In [0]:
text_file = sc.textFile('/FileStore/tables/test1.txt')

In [0]:
text_file.collect()

Out[2]: ['Apache Spark Examples.',
 'These examples give a quick overview of the Spark API. ',
 'Spark is built on the concept of distributed datasets, which contain arbitrary Java or Python objects. You create a dataset from external data, then apply parallel operations to it. The building block of the Spark API is its RDD API. In the RDD API, there are two types of operations: transformations, which define a new dataset based on previous ones, and actions, which kick off a job to execute on a cluster. On top of Spark’s RDD API, high level APIs are provided, e.g. DataFrame API and Machine Learning API. These high level APIs provide a concise way to conduct certain data operations. In this page, we will show examples using RDD API as well as examples using high level APIs.']

In [0]:
l = [('Ankit', 25), ('Jalfaizy', 22), ('saurabh', 20), ('Bala', 26)]
rdd = sc.parallelize(l)
print(rdd.collect())

[('Ankit', 25), ('Jalfaizy', 22), ('saurabh', 20), ('Bala', 26)]

In [0]:
def computeContribs(urls, rank) :
  num_urls = len(urls)
  for url in urls :
    yield(url, rank / num_urls)

mapLink = sc.parallelize([["MapR","Baidu"], ["MapR", "Blogger"], ["Baidu", "MapR"], ["Blogger", "Google"], ["Blogger", "Baidu"], ["Google", "MapR"]])
links = mapLink.groupByKey()

ranks = links.map(lambda pairs : (pairs[0], 1))
print("initial ranks : ", ranks.collect())

for i in range(5) :
  cvalues = links.join(ranks)
  contribs = links.join(ranks).flatMap(
                lambda url_urls_rank : computeContribs(url_urls_rank[1][0], url_urls_rank[1][1]))
  ranks = contribs.reduceByKey(lambda x, y : x + y).mapValues(lambda rank : 0.15 + 0.85*rank)
  print("ranks(",i,"):",ranks.collect())
  print()

initial ranks : [('Google', 1), ('MapR', 1), ('Baidu', 1), ('Blogger', 1)]
ranks( 0 ): [('Google', 0.575), ('MapR', 1.8499999999999999), ('Baidu', 1.0), ('Blogger', 0.575)]

ranks( 1 ): [('MapR', 1.4887499999999998), ('Blogger', 0.9362499999999999), ('Baidu', 1.1806249999999998), ('Google', 0.394375)]

ranks( 2 ): [('Google', 0.5479062499999999), ('MapR', 1.4887499999999996), ('Baidu', 1.1806249999999998), ('Blogger', 0.7827187499999999)]

ranks( 3 ): [('MapR', 1.6192515624999995), ('Baidu', 1.1153742187499998), ('Blogger', 0.7827187499999998), ('Google', 0.48265546875)]

ranks( 4 ): [('MapR', 1.5083252343749995), ('Google', 0.48265546874999987), ('Blogger', 0.8381819140624998), ('Baidu', 1.1708373828124996)]

In [0]:
from pyspark.sql import Row
l = [('Ankit', 25), ('Jalfaizy', 22), ('saurabh', 20), ('Bala', 26)]
rdd = sc.parallelize(l)
people = rdd.map(lambda x : Row(name=x[0], age=int(x[1])))
schemaPeople = sqlContext.createDataFrame(people)           

In [0]:
print(type(schemaPeople))
schemaPeople.printSchema()
print(schemaPeople.collect())
print(schemaPeople.count())

<class 'pyspark.sql.dataframe.DataFrame'>
root
-- name: string (nullable = true)
-- age: long (nullable = true)

[Row(name='Ankit', age=25), Row(name='Jalfaizy', age=22), Row(name='saurabh', age=20), Row(name='Bala', age=26)]
4

In [0]:
%fs ls

path,name,size
dbfs:/FileStore/,FileStore/,0
dbfs:/count/,count/,0
dbfs:/databricks-datasets/,databricks-datasets/,0
dbfs:/databricks-results/,databricks-results/,0
dbfs:/tmp/,tmp/,0


In [0]:
peopleDF = spark.read.parquet("/FileStore/tables/people-10m.parquet")
peopleDF.printSchema()

root
-- id: integer (nullable = true)
-- firstName: string (nullable = true)
-- middleName: string (nullable = true)
-- lastName: string (nullable = true)
-- gender: string (nullable = true)
-- birthDate: timestamp (nullable = true)
-- ssn: string (nullable = true)
-- salary: integer (nullable = true)

In [0]:
peopleDF.show(5)

+---+---------+----------+----------+------+-------------------+-----------+------+
 id|firstName|middleName| lastName|gender| birthDate| ssn|salary|
+---+---------+----------+----------+------+-------------------+-----------+------+
 1| Pennie| Carry|Hirschmann| F|1955-07-02 04:00:00|981-43-9345| 56172|
 2| An| Amira| Cowper| F|1992-02-08 05:00:00|978-97-8086| 40203|
 3| Quyen| Marlen| Dome| F|1970-10-11 04:00:00|957-57-8246| 53417|
 4| Coralie| Antonina| Marshal| F|1990-04-11 04:00:00|963-39-4885| 94727|
 5| Terrie| Wava| Bonar| F|1980-01-16 05:00:00|964-49-8051| 79908|
+---+---------+----------+----------+------+-------------------+-----------+------+
only showing top 5 rows

In [0]:
peopleDF.head(5)

Out[27]: [Row(id=1, firstName='Pennie', middleName='Carry', lastName='Hirschmann', gender='F', birthDate=datetime.datetime(1955, 7, 2, 4, 0), ssn='981-43-9345', salary=56172),
 Row(id=2, firstName='An', middleName='Amira', lastName='Cowper', gender='F', birthDate=datetime.datetime(1992, 2, 8, 5, 0), ssn='978-97-8086', salary=40203),
 Row(id=3, firstName='Quyen', middleName='Marlen', lastName='Dome', gender='F', birthDate=datetime.datetime(1970, 10, 11, 4, 0), ssn='957-57-8246', salary=53417),
 Row(id=4, firstName='Coralie', middleName='Antonina', lastName='Marshal', gender='F', birthDate=datetime.datetime(1990, 4, 11, 4, 0), ssn='963-39-4885', salary=94727),
 Row(id=5, firstName='Terrie', middleName='Wava', lastName='Bonar', gender='F', birthDate=datetime.datetime(1980, 1, 16, 5, 0), ssn='964-49-8051', salary=79908)]

In [0]:
print(peopleDF.count())
print(peopleDF.columns)
print(peopleDF.describe().show(5))

10000000
['id', 'firstName', 'middleName', 'lastName', 'gender', 'birthDate', 'ssn', 'salary']

In [0]:
peopleDF.select('firstName', 'birthDate').show(5)

+---------+-------------------+
firstName| birthDate|
+---------+-------------------+
 Pennie|1955-07-02 04:00:00|
 An|1992-02-08 05:00:00|
 Quyen|1970-10-11 04:00:00|
 Coralie|1990-04-11 04:00:00|
 Terrie|1980-01-16 05:00:00|
+---------+-------------------+
only showing top 5 rows

In [0]:
peopleDF.select('gender').distinct().count()

Out[32]: 2

In [0]:
peopleDF.filter(peopleDF['gender'] == 'F').count()

Out[33]: 5187302

In [0]:
peopleDF.filter(peopleDF.gender == 'F').count()

Out[34]: 5187302

In [0]:
peopleDF.filter("gender = 'F'").count()

Out[36]: 5187302

In [0]:
from pyspark.sql.functions import year
peopleDF.filter(year("birthDate") > "1955").count()

Out[37]: 9168285

In [0]:
test1 = peopleDF.select('firstName').filter("gender='M'" and year("birthDate") > '1960')
test1.show(5)

+---------+
firstName|
+---------+
 An|
 Quyen|
 Coralie|
 Terrie|
 Chassidy|
+---------+
only showing top 5 rows

In [0]:
peopleDF.select("firstName", year("birthDate")).filter("gender='M'" and year("birthDate") > "1960").show(10)

+---------+---------------+
firstName|year(birthDate)|
+---------+---------------+
 An| 1992|
 Quyen| 1970|
 Coralie| 1990|
 Terrie| 1980|
 Chassidy| 1990|
 Geri| 1970|
 Patria| 1985|
 Terese| 1967|
 Wava| 1963|
 Sophie| 1979|
+---------+---------------+
only showing top 10 rows

In [0]:
peopleDF.groupBy("Gender").count().show()

+------+-------+
Gender| count|
+------+-------+
 F|5187302|
 M|4812698|
+------+-------+

In [0]:
peopleDF.groupBy("firstName").count().orderBy("firstName").show(5)

+---------+-----+
firstName|count|
+---------+-----+
 Aaron| 4020|
 Abbey| 1273|
 Abbie| 1288|
 Abby| 1235|
 Abdul| 3966|
+---------+-----+
only showing top 5 rows